In [16]:
import numpy as np
import os
import pandas as pd

## Предобработка

In [17]:
import os
import warnings
warnings.filterwarnings('ignore')

In [18]:
from IPython.display import display, clear_output

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

In [19]:
interactions_df = pd.read_csv('data_original/interactions.csv')
users_df = pd.read_csv('data_original/users.csv')
items_df = pd.read_csv('data_original/items.csv')

In [20]:
interactions_df = interactions_df[interactions_df['last_watch_dt'] < '2021-04-01']

In [21]:
users_interactions_count_df = interactions_df.groupby(['user_id', 'item_id']).size().groupby('user_id').size()
print('# users: %d' % len(users_interactions_count_df))
users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['user_id']]
print('# users with at least 5 interactions: %d' % len(users_with_enough_interactions_df))

# users: 86614
# users with at least 5 interactions: 14563


In [22]:
print('# of interactions: %d' % len(interactions_df))
interactions_from_selected_users_df = interactions_df.merge(users_with_enough_interactions_df, 
               how = 'right',
               left_on = 'user_id',
               right_on = 'user_id')
print('# of interactions from users with at least 5 interactions: %d' % len(interactions_from_selected_users_df))

# of interactions: 263874
# of interactions from users with at least 5 interactions: 142670


In [23]:

import math
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = interactions_from_selected_users_df \
                    .groupby(['user_id', 'item_id'])['watched_pct'].sum() \
                    .apply(smooth_user_preference).reset_index()
print('# of unique user/item interactions: %d' % len(interactions_full_df))
interactions_full_df.head(10)

# of unique user/item interactions: 142670


,user_id,item_id,watched_pct
0,21,849,6.375039
1,21,4345,6.658211
2,21,10283,6.658211
3,21,12261,6.658211
4,21,15997,6.658211
5,32,952,6.044394
6,32,4382,4.954196
7,32,4807,6.658211
8,32,10436,6.658211
9,32,12132,6.658211


In [24]:
interactions_train_df, interactions_test_df = train_test_split(interactions_full_df,
                                   stratify=interactions_full_df['user_id'], 
                                   test_size=0.20,
                                   random_state=42)

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

# interactions on Train set: 114136
# interactions on Test set: 28534


In [25]:
interactions_full_indexed_df = interactions_full_df.set_index('user_id')
interactions_train_indexed_df = interactions_train_df.set_index('user_id')
interactions_test_indexed_df = interactions_test_df.set_index('user_id')

In [26]:
def get_items_interacted(person_id, interactions_df):
    # Get the user's data and merge in the movie information.
    interacted_items = interactions_df.loc[person_id]['item_id']
    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])

In [27]:
from IPython.display import display, clear_output

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
# Constants
SEED = 42 # random seed for reproducibility
LR = 1e-3 # learning rate, controls the speed of the training
WEIGHT_DECAY = 0.01 # lambda for L2 reg. ()
NUM_EPOCHS = 100 # num training epochs (how many times each instance will be processed)
GAMMA = 0.9 # learning rate scheduler parameter
BATCH_SIZE = 6000 # training batch size
EVAL_BATCH_SIZE = 6000 # evaluation batch size.
DEVICE = 'cuda' #'cuda' # device to make the calculations on

In [28]:
total_df = pd.concat([interactions_train_df, interactions_test_indexed_df.reset_index()], ignore_index=True)
total_df['user_id'], users_keys = total_df.user_id.factorize()
total_df['item_id'], items_keys = total_df.item_id.factorize()

train_encoded = total_df.iloc[:len(interactions_train_df)].values
test_encoded = total_df.iloc[len(interactions_train_df):].values

In [29]:
items_keys

Index([ 1554,  2360, 12544,  1358,  4807,  7134,  1106,   758,  3042,  5984,
       ...
       14137,  7655,  1924,  7720,  5437, 14644,   204, 13711,   115, 16422],
      dtype='int64', length=8287)

In [30]:
interactions_full_indexed_df

,item_id,watched_pct
user_id,,
21,849,6.375039
21,4345,6.658211
21,10283,6.658211
21,12261,6.658211
21,15997,6.658211
...,...,...
1097530,849,5.129283
1097530,4155,0.000000
1097530,8386,6.658211


In [31]:
from scipy.sparse import csr_matrix
shape = [int(total_df['user_id'].max()+1), int(total_df['item_id'].max()+1)]
X_train = csr_matrix((train_encoded[:, 2], (train_encoded[:, 0], train_encoded[:, 1])), shape=shape).toarray()
X_test = csr_matrix((test_encoded[:, 2], (test_encoded[:, 0], test_encoded[:, 1])), shape=shape).toarray()

In [32]:
total_encoded = total_df.iloc[:len(interactions_train_df)].values
X = csr_matrix((train_encoded[:, 2], (train_encoded[:, 0], train_encoded[:, 1])), shape=shape).toarray()

In [33]:
class UserOrientedDataset(Dataset):
    def __init__(self, X):
        super().__init__()
        self.X = X.astype(np.float32)
        self.len = len(X)

    def __len__(self):
        return self.len

    def __getitem__(self, index):
        return self.X[index]

In [34]:
train_dl = DataLoader(
    UserOrientedDataset(X_train),
    batch_size = BATCH_SIZE,
    shuffle = True
)

test_dl = DataLoader(
    UserOrientedDataset(X_test),
    batch_size = EVAL_BATCH_SIZE,
    shuffle = False
)

dls = {'train': train_dl, 'test': test_dl}

# Добавил блок автоэнкодера, нормализацию и dropout и функцию активации SiLU т.к она показала лучшее качество

In [35]:
class Model(nn.Module):
    def __init__(self, in_and_out_features=8287):
        super().__init__()
        self.in_and_out_features = in_and_out_features
        self.hidden_size1 = 2000
        self.hidden_size2 = 2000

        self.sequential = nn.Sequential(
        nn.Linear(in_and_out_features, self.hidden_size1),
        nn.BatchNorm1d(self.hidden_size1),
        nn.SiLU(),
        nn.Dropout(0.1),  # Dropout layer after the first linear layer with a dropout rate of 0.5
        nn.Linear(self.hidden_size1, self.hidden_size2),
        nn.BatchNorm1d(self.hidden_size2),
        nn.SiLU(),
        nn.Dropout(0.1),  # Dropout layer after the second linear layer with a dropout rate of 0.3
        nn.Linear(self.hidden_size2, in_and_out_features)
        )

    def forward(self, x):
        x = self.sequential(x)
        return x

In [36]:
torch.manual_seed(SEED) # Fix random seed to have reproducible weights of model layers

model = Model()
model.to(DEVICE)

# Initialize GD method, which will update the weights of the model
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
# Initialize learning rate scheduler, which will decrease LR according to some rule
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=LR, total_steps=4000, div_factor=25, pct_start=0.2)

def rmse_for_sparse(x_pred, x_true):
    mask = (x_true > 0)
    sq_diff = (x_pred * mask - x_true) ** 2
    mse = sq_diff.sum() / mask.sum()
    return mse ** (1/2)

In [37]:
metrics_dict = {
    "Epoch": [],
    "Train RMSE": [],
    "Test RMSE": [],
}

# Train loop
for epoch in range(NUM_EPOCHS):
    metrics_dict["Epoch"].append(epoch)
    for stage in ['train', 'test']:
        with torch.set_grad_enabled(stage == 'train'): # Whether to start building a graph for a backward pass
            if stage == 'train':
                model.train() # Enable some "special" layers (will speak about later)
            else:
                model.eval() # Disable some "special" layers (will speak about later)

            loss_at_stage = 0 
            for batch in dls[stage]:
                batch = batch.to(DEVICE)
                x_pred = model(batch) # forward pass: model(x_batch) -> calls forward()
                loss = rmse_for_sparse(x_pred, batch) # ¡Important! y_pred is always the first arg
                if stage == "train":
                    loss.backward() # Calculate the gradients of all the parameters wrt loss
                    optimizer.step() # Update the parameters
                    scheduler.step()
                    optimizer.zero_grad() # Zero the saved gradient
                loss_at_stage += loss.item() * len(batch)
            rmse_at_stage = (loss_at_stage / len(dls[stage].dataset)) ** (1/2)
            metrics_dict[f"{stage.title()} RMSE"].append(rmse_at_stage)
            
    if (epoch == NUM_EPOCHS - 1) or epoch % 10 == 9:
        clear_output(wait=True)
        display(pd.DataFrame(metrics_dict))

,Epoch,Train RMSE,Test RMSE
0,0,2.334902,2.332204
1,1,2.291628,2.331182
2,2,2.251149,2.329684
3,3,2.211368,2.327409
4,4,2.171554,2.324048
...,...,...,...
95,95,0.578895,1.295480
96,96,0.583493,1.297303
97,97,0.593253,1.311995
98,98,0.583782,1.313170


In [38]:
with torch.no_grad():
    X_pred = model(torch.Tensor(X_test).to(DEVICE))
X_pred

tensor([[ 8.8101e-01,  1.0371e+00,  1.0955e+00,  ..., -2.2062e-01,
          1.6884e-01, -2.7020e-01],
        [-2.4114e-01,  1.9255e-01, -1.4084e-01,  ..., -1.3129e-01,
         -3.1809e-02,  1.6262e-02],
        [-3.0018e-03,  2.4590e-01,  1.1676e-01,  ..., -2.6522e-01,
          2.3115e-01, -1.8103e-01],
        ...,
        [-1.0334e-01,  2.5054e-01, -4.4833e-02,  ..., -1.1341e-01,
          2.7182e-02, -9.9426e-02],
        [ 1.5708e-01,  6.0678e-01,  8.4680e-02,  ..., -9.5058e-02,
          1.1787e-01, -3.8322e-02],
        [ 2.1550e-01,  2.9218e-01, -7.0731e-04,  ...,  6.2704e-02,
          1.0136e-01,  4.5902e-02]], device='cuda:0')

In [39]:


import numpy as np
from sklearn.metrics import average_precision_score

class AERecommender:
    
    MODEL_NAME = 'Autoencoder'
    
    def __init__(self, X_preds, X_train_and_val, X_test):

        self.X_preds = X_preds.cpu().detach().numpy()
        self.X_train_and_val = X_train_and_val
        self.X_test = X_test
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_select_idx, topn=10, verbose=False):
        user_preds = self.X_preds[user_id][items_to_select_idx]
        items_idx = items_to_select_idx[np.argsort(-user_preds)[:topn]]

        # Recommend the highest predicted rating movies that the user hasn't seen yet.
        return items_idx

    def evaluate(self, size=100):

        X_total = self.X_train_and_val + self.X_test

        true_5 = []
        true_10 = []
        map10 = []
        for user_id in range(len(self.X_test)):
            interacted_items = np.argwhere(X_total[user_id] > 0).ravel()
            
            non_zero = np.argwhere(self.X_test[user_id] > 0).ravel()
            select_from = np.setdiff1d(np.arange(X_total.shape[1]), interacted_items)
            
            for non_zero_idx in non_zero:
                random_non_interacted_100_items = np.random.choice(select_from, size=20, replace=False) 
                preds = self.recommend_items(user_id, np.append(random_non_interacted_100_items, non_zero_idx), topn=10)
                
                ap = average_precision_score(np.isin(preds, non_zero), np.arange(1, 11)) 
                map10.append(ap)
                
        mean_ap10 = np.mean(map10)
        
        return {"recall@5": np.mean(true_5), 
                "recall@10": np.mean(true_10),
                "map@10": mean_ap10}

In [40]:
ae_recommender_model = AERecommender(X_pred, X_train, X_train)
ae_global_metrics = ae_recommender_model.evaluate()
ae_global_metrics

{'recall@5': nan, 'recall@10': nan, 'map@10': 0.15695218963476443}

# Получение предсказаний для сервиса

In [42]:
with torch.no_grad():
    full_pred = model(torch.Tensor(UserOrientedDataset(X)).to(DEVICE)).cpu()

In [43]:
ae_recommender_model = AERecommender(X_pred, X, X)
ae_global_metrics = ae_recommender_model.evaluate()
ae_global_metrics

{'recall@5': nan, 'recall@10': nan, 'map@10': 0.1578540718688651}

In [58]:
total_encoded = total_df.iloc[:len(interactions_train_df)].values
X = csr_matrix((train_encoded[:, 2], (train_encoded[:, 0], train_encoded[:, 1])), shape=shape).toarray()

In [59]:
with torch.no_grad():
    full_pred = model(torch.Tensor(UserOrientedDataset(X)).to(DEVICE)).cpu()

In [60]:
# восстанавливаем оригинальные индексы пользователей и айтемов
users = pd.Series(range(len(X)), index=users_keys)  
items = pd.Series(range(len(X[0])), index=items_keys)

# преобразуем предсказания в dataframe
preds_df = pd.DataFrame(full_pred.numpy(), columns=items.index, index=users.index)

# стек данных в длинный формат  
preds_long_df = preds_df.stack().reset_index()
preds_long_df.columns = ['user_id', 'item_id', 'prediction']

# фильтруем топ предсказаний для каждого пользователя  
top_predictions = (preds_long_df.groupby('user_id')
                   .apply(lambda x: x.nlargest(10, 'prediction')))
                   
top_predictions = top_predictions.reset_index(drop=True)
top_predictions = pd.DataFrame(top_predictions)
top_predictions=top_predictions.drop('prediction',axis=1)

In [61]:
top_predictions.groupby('user_id').agg({'item_id': list}).to_json('autoencoder.json')